In [1]:
%matplotlib inline
import utils; reload(utils)
from random import randint
from utils import *

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
path = 'data/fishes/'
full_path = os.getcwd() + '/' + path

In [3]:
batch_size = 64

In [4]:
VERSION = 'sgd_002'

### To do list:

1. Create validation set and sample
2. Move to separate dirs for each set
3. Finetune and train
4. Submit to kaggle

In [5]:
classes = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']

# Finetune and train

In [14]:
from vgg16bn import Vgg16BN
model = vgg_ft_bn(8)

In [15]:
from vgg16 import Vgg16
model = vgg_ft(8)

In [20]:
img_gen = image.ImageDataGenerator(
    rotation_range=30,
    horizontal_flip=True,
    zoom_range=0.4,
)

In [21]:
batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', gen=img_gen, batch_size=1, shuffle=False)

(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 2573 images belonging to 8 classes.
Found 563 images belonging to 8 classes.
Found 2573 images belonging to 8 classes.
Found 563 images belonging to 8 classes.
Found 154 images belonging to 1 classes.


In [26]:
test_batches = get_batches(path+'test_valid', shuffle=False, batch_size=1)
test_labels = onehot(test_batches.classes)

Found 641 images belonging to 8 classes.


In [84]:
len(val_labels)

563

In [56]:
img_gen = image.ImageDataGenerator(
    rotation_range=30,
    horizontal_flip=True,
    zoom_range=0.4,
    #data_format='channels_first'
)
batches = img_gen.flow_from_directory(
    path+'test_aug', 
    target_size=(224,224),
    class_mode='categorical',
    shuffle=False,
    batch_size=4,
    save_to_dir=path+'augmentation',
)

Found 1 images belonging to 1 classes.


In [73]:
batches.next()

(array([[[[  88.,   88.,   99., ...,   95.,   94.,   93.],
          [  82.,   88.,   88., ...,   78.,   94.,   93.],
          [  77.,   82.,   88., ...,   78.,   93.,   70.],
          ..., 
          [  74.,   67.,   73., ...,   46.,   46.,   46.],
          [  74.,   67.,   90., ...,   46.,   46.,   46.],
          [  67.,   73.,   90., ...,   46.,   46.,   46.]],
 
         [[  71.,   71.,   69., ...,  129.,  128.,  127.],
          [  65.,   71.,   71., ...,  112.,  128.,  127.],
          [  60.,   65.,   71., ...,  112.,  127.,  106.],
          ..., 
          [  74.,   67.,   73., ...,   75.,   75.,   75.],
          [  74.,   67.,   90., ...,   75.,   75.,   75.],
          [  67.,   73.,   90., ...,   75.,   75.,   75.]],
 
         [[  64.,   64.,   67., ...,  138.,  137.,  136.],
          [  58.,   64.,   64., ...,  121.,  137.,  136.],
          [  53.,   58.,   64., ...,  121.,  136.,  118.],
          ..., 
          [  66.,   59.,   65., ...,   89.,   89.,   89.],
  

### Training the model

In [75]:
trn = get_data(path + 'train')
val = get_data(path + 'valid')

Found 2573 images belonging to 8 classes.
Found 563 images belonging to 8 classes.


In [76]:
len(trn), len(val)

(5146, 1126)

In [10]:
test = get_data(path+'test_valid')

Found 641 images belonging to 8 classes.


In [24]:
test_batches = get_batches(path+'test_v2', batch_size=batch_size, shuffle=False)


Found 12153 images belonging to 1 classes.


In [95]:
curr_size = 0
total_size = float(len(test_batches.filenames))
for x in test_batches:
    curr_size += len(x[0])
    print(curr_size/total_size)
    if curr_size >= total_size:
        break

0.00526618941825
0.0105323788365
0.0157985682548
0.021064757673
0.0263309470913
0.0315971365095
0.0368633259278
0.042129515346
0.0473957047643
0.0526618941825
0.0579280836008
0.063194273019
0.0684604624373
0.0737266518555
0.0789928412738
0.084259030692
0.0895252201103
0.0947914095285
0.100057598947
0.105323788365
0.110589977783
0.115856167202
0.12112235662
0.126388546038
0.131654735456
0.136920924875
0.142187114293
0.147453303711
0.152719493129
0.157985682548
0.163251871966
0.168518061384
0.173784250802
0.179050440221
0.184316629639
0.189582819057
0.194849008475
0.200115197894
0.205381387312
0.21064757673
0.215913766148
0.221179955567
0.226446144985
0.231712334403
0.236978523821
0.24224471324
0.247510902658
0.252777092076
0.258043281494
0.263309470913
0.268575660331
0.273841849749
0.279108039167
0.284374228586
0.289640418004
0.294906607422
0.30017279684
0.305438986259
0.310705175677
0.315971365095
0.321237554513


KeyboardInterrupt: 

In [108]:
save_array(path+'results/trn.dat', trn)
save_array(path+'results/val.dat', val)

In [109]:
save_array(path+'results/test_valid.dat', test)

Load data from files

In [64]:
trn = load_array(path + 'results/trn.dat')

In [65]:
len(trn)

2573

In [11]:
val = load_array(path + 'results/val.dat')

In [12]:
test = load_array(path + 'results/test_valid.dat')

### Precompute convolutional output

We pre-compute the output

In [16]:
conv_layers, fc_layers = split_at(model, Convolution2D)

In [78]:
conv_model = Sequential(conv_layers)

In [18]:
_, old_fc_layers = split_at(old_model, Convolution2D)

In [80]:
old_fc_layers

In [20]:
conv_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_2 (Lambda)                (None, 3, 224, 224)   0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
zeropadding2d_14 (ZeroPadding2D) (None, 3, 226, 226)   0           lambda_2[0][0]                   
                                                                   lambda_2[0][0]                   
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 64, 224, 224)  0           zeropadding2d_14[0][0]           
                                                                   zeropadding2d_14[1][0]           
___________________________________________________________________________________________

In [157]:
#old_conv_layers, old_fc_layers = split_at(old_model, Convolution2D)
#old_conv_model = Sequential(old_conv_layers)

In [158]:
#old_conv_model.summary()

In [79]:
aug_conv_feat = conv_model.predict(trn)
aug_conv_val_feat = conv_model.predict(val)

In [160]:
#old_conv_feat = old_conv_model.predict(trn)
#old_conv_val_feat = old_conv_model.predict(val)

In [161]:
conv_test_feat = conv_model.predict(test)

In [115]:
#old_conv_test_feat = old_conv_model.predict(test)

In [80]:
save_array(path + 'results/aug_conv_val_feat.dat', aug_conv_val_feat)
save_array(path + 'results/aug_conv_feat.dat', aug_conv_feat)

In [163]:
save_array(path + 'results/conv_test_feat.dat', conv_test_feat)

### Load precomputed conv. output

In [8]:
conv_feat = load_array(path+'results/conv_feat.dat')
conv_val_feat = load_array(path+'results/conv_val_feat.dat')
conv_test_feat = load_array(path+'results/conv_test_feat.dat')

In [21]:
old_conv_feat = load_array(path+'results/old_conv_feat.dat')
old_conv_val_feat = load_array(path+'results/old_conv_val_feat.dat')
old_conv_test_feat = load_array(path+'results/old_conv_test_feat.dat')

KeyboardInterrupt: 

In [9]:
conv_val_feat.shape

(563, 512, 14, 14)

# Train model


In [28]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        BatchNormalization(axis=1),
        #Dropout(p/4),
        Flatten(),
        Dense(4096, activation='relu'),
        #BatchNormalization(),
        #Dropout(p),
        Dense(4096, activation='relu'),
        #BatchNormalization(),
        #Dropout(p/2),
        Dense(8, activation='softmax')
    ]

In [29]:
p = 0.5  # dropout

In [30]:
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(SGD(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [31]:
bn_model.fit(
    conv_feat, #conv_feat,
    trn_labels,
    batch_size=batch_size,
    nb_epoch=3,
    validation_data=(conv_val_feat, val_labels))

Train on 2573 samples, validate on 563 samples
Epoch 1/3
2573/2573 [==============================] - 2s - loss: 1.0855 - acc: 0.7209 - val_loss: 0.4019 - val_acc: 0.8508
Epoch 2/3
2573/2573 [==============================] - 2s - loss: 0.1388 - acc: 0.9755 - val_loss: 0.3401 - val_acc: 0.8845
Epoch 3/3
2573/2573 [==============================] - 2s - loss: 0.0627 - acc: 0.9953 - val_loss: 0.1842 - val_acc: 0.9449


In [32]:
bn_model.optimizer.lr = 0.001

In [33]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=7, 
             validation_data=(conv_val_feat, val_labels))

Train on 2573 samples, validate on 563 samples
Epoch 1/7
2573/2573 [==============================] - 2s - loss: 0.0354 - acc: 0.9992 - val_loss: 0.1695 - val_acc: 0.9449
Epoch 2/7
2573/2573 [==============================] - 2s - loss: 0.0253 - acc: 0.9996 - val_loss: 0.1678 - val_acc: 0.9467
Epoch 3/7
2573/2573 [==============================] - 2s - loss: 0.0193 - acc: 1.0000 - val_loss: 0.1593 - val_acc: 0.9485
Epoch 4/7
2573/2573 [==============================] - 2s - loss: 0.0155 - acc: 1.0000 - val_loss: 0.1589 - val_acc: 0.9467
Epoch 5/7
2573/2573 [==============================] - 2s - loss: 0.0129 - acc: 1.0000 - val_loss: 0.1567 - val_acc: 0.9503
Epoch 6/7
2573/2573 [==============================] - 2s - loss: 0.0110 - acc: 1.0000 - val_loss: 0.1550 - val_acc: 0.9503
Epoch 7/7
2573/2573 [==============================] - 2s - loss: 0.0096 - acc: 1.0000 - val_loss: 0.1527 - val_acc: 0.9485


In [34]:
#bn_model.save_weights(path+'models/old_conv_512_6_{}_4096_no_dropout.h5'.format(VERSION))

In [35]:
bn_model.evaluate(conv_test_feat, test_labels)

640/641 [============================>.] - ETA: 0s

[0.15147959156646196, 0.95319812792511704]

In [89]:
bn_model.load_weights(path+'models/conv_512_6_{}.h5'.format(VERSION))

# Submit

In [92]:
test = load_array(path+'results/test.dat')

In [65]:
#test_batches = get_batches(path + 'test', shuffle=False)
preds = bn_model.predict(old_conv_test_feat, batch_size=batch_size)

In [ ]:
test_batches.

In [66]:
def do_clip(arr, mx):
    return np.clip(arr, (1-mx)/7, mx)

In [67]:
subm = do_clip(preds, 0.85)
raw_test_filenames = [f.split('/')[-1] for f in test_filenames]


In [68]:
subm_name = path+'results/subm_bb_dp_{}.gz'.format(VERSION)

In [70]:
raw_test_filenames

['image_12090.jpg',
 'image_12064.jpg',
 'image_12098.jpg',
 'image_12052.jpg',
 'image_12055.jpg',
 'image_12107.jpg',
 'image_12135.jpg',
 'image_12073.jpg',
 'image_12071.jpg',
 'image_12002.jpg',
 'image_12016.jpg',
 'image_12007.jpg',
 'image_12099.jpg',
 'image_12013.jpg',
 'image_12106.jpg',
 'image_12022.jpg',
 'image_12111.jpg',
 'image_12078.jpg',
 'image_12091.jpg',
 'image_12146.jpg',
 'image_12057.jpg',
 'image_12030.jpg',
 'image_12026.jpg',
 'image_12074.jpg',
 'image_12120.jpg',
 'image_12000.jpg',
 'image_12108.jpg',
 'image_12011.jpg',
 'image_12036.jpg',
 'image_12039.jpg',
 'image_12070.jpg',
 'image_12015.jpg',
 'image_12008.jpg',
 'image_12001.jpg',
 'image_12093.jpg',
 'image_12121.jpg',
 'image_12035.jpg',
 'image_12119.jpg',
 'image_12133.jpg',
 'image_12080.jpg',
 'image_12136.jpg',
 'image_12004.jpg',
 'image_12019.jpg',
 'image_12032.jpg',
 'image_12047.jpg',
 'image_12089.jpg',
 'image_12009.jpg',
 'image_12048.jpg',
 'image_12141.jpg',
 'image_12140.jpg',


In [69]:
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'image', raw_test_filenames)
submission.head()

ValueError: Length of values does not match length of index

In [98]:
submission.to_csv(subm_name, index=False, compression='gzip')

In [99]:
FileLink(subm_name)

/home/ubuntu/nbs/data/fishes/results/subm_bb_dp_sgd_002.gz